In [1]:
!pip install bayesian-optimization
!pip install tensorflow

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=c6871f46bdfe6dfb54c8d6dd89df2b4f1206972e3ddc48f858fe2a176429e684
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization
     |████████████████████████████████| 462 kB 5.1 MB/s 


In [9]:
from bayes_opt import BayesianOptimization
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models, callbacks

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt


from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier

import time


import warnings

In [3]:
warnings.filterwarnings('ignore')
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [4]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

170508288/170498071 [==============================] - 2s 0us/step


In [14]:
# Set hyperparameter space
hyperspace_cnn = {
    'conv1_act': (0, 7),
    'conv2_act': (0, 7),
    'conv3_act': (0, 7),
    'conv1_ker': (2, 10),
    'conv2_ker': (2, 10),
    'conv3_ker': (2, 10),
    'conv1_filter': (20, 32),
    'conv2_filter': (20, 32),
    'conv3_filter': (50, 100),
    'pool1_ker': (2, 4),
    'pool2_ker': (2, 4),
    'pool3_ker': (2, 4),
    'dense1_neuron': (32, 64),
    'dense2_neuron': (32, 64),
    'dense1_act': (0, 7),
    'dense2_act': (0, 7),
    'dropout_rate':(0, 0.5),
    'learning_rate':(0.0005, 0.002),
    'beta_1':(0.9,1),
    'beta_2':(0.95,1),
 }

In [24]:
def bo_tune_cnn(conv1_act,conv2_act,conv3_act,conv1_ker,conv2_ker,conv3_ker,conv1_filter,conv2_filter,conv3_filter,
                pool1_ker,pool2_ker,pool3_ker,dense1_act,dense2_act,dense1_neuron,dense2_neuron,dropout_rate,learning_rate,beta_1,beta_2):

    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu','elu', 'exponential']
    
    conv1_act = activationL[int(conv1_act)]
    conv2_act = activationL[int(conv2_act)]
    conv3_act = activationL[int(conv3_act)]
    dense1_act = activationL[int(dense1_act)]
    dense2_act = activationL[int(dense2_act)]

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate,beta_1=beta_1,beta_2=beta_2)
    
    

    model = models.Sequential()
    model.add(layers.Conv2D(int(conv1_filter), int(conv1_ker), activation=conv1_act, padding="same", input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D(int(pool1_ker)))
    model.add(layers.Conv2D(int(conv2_filter), int(conv2_ker), activation=conv2_act, padding="same"))
    model.add(layers.MaxPooling2D(int(pool2_ker)))
    model.add(layers.Conv2D(int(conv3_filter), int(conv3_ker), activation=conv3_act, padding="same"))
    model.add(layers.MaxPooling2D(int(pool3_ker)))
    model.add(layers.Flatten())
    model.add(layers.Dense(int(dense1_neuron), activation=dense1_act))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(int(dense2_neuron), activation=dense2_act))
    model.add(layers.Dense(10))
    
    model.compile(optimizer = opt,
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    
    es = callbacks.EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=5)


    history = model.fit(train_images, train_labels, epochs = 5, validation_split = 0.1, callbacks=[es])
    
    return -1.0 * min(history.history['val_loss'])

In [25]:
def run_cnn(best_param):
        conv1_ker = best_param['conv1_ker']
        conv2_ker = best_param['conv2_ker']
        conv3_ker = best_param['conv3_ker']
        conv1_filter = best_param['conv1_filter']
        conv2_filter = best_param['conv2_filter']
        conv3_filter = best_param['conv3_filter']
        pool1_ker = best_param['pool1_ker']
        pool2_ker = best_param['pool2_ker']
        pool3_ker = best_param['pool3_ker']
        dense1_neuron = best_param['dense1_neuron']
        dense2_neuron = best_param['dense2_neuron']
        dropout_rate = best_param['dropout_rate']
        learning_rate = best_param['learning_rate']
        beta_1 = best_param['beta_1']
        beta_2 = best_param['beta_2']


        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate,beta_1=beta_1,beta_2=beta_2)

                        
        activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu','elu', 'exponential']  
        conv1_act = activationL[int(best_param['conv1_act'])]
        conv2_act = activationL[int(best_param['conv2_act'])]
        conv3_act = activationL[int(best_param['conv3_act'])]
        dense1_act = activationL[int(best_param['dense1_act'])]
        dense2_act = activationL[int(best_param['dense2_act'])]

        model = models.Sequential()
        model.add(layers.Conv2D(int(conv1_filter), int(conv1_ker), activation=conv1_act, padding="same", input_shape=(32, 32, 3)))
        model.add(layers.MaxPooling2D(int(pool1_ker)))
        model.add(layers.Conv2D(int(conv2_filter), int(conv2_ker), activation=conv2_act, padding="same"))
        model.add(layers.MaxPooling2D(int(pool2_ker)))
        model.add(layers.Conv2D(int(conv3_filter), int(conv3_ker), activation=conv3_act, padding="same"))
        model.add(layers.MaxPooling2D(int(pool3_ker)))
        model.add(layers.Flatten())
        model.add(layers.Dense(int(dense1_neuron), activation=dense1_act))
        model.add(layers.Dropout(dropout_rate))
        model.add(layers.Dense(int(dense2_neuron), activation=dense2_act))
        model.add(layers.Dense(10))

        model.compile(optimizer = opt,
                      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])


        es = callbacks.EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=5)


        history = model.fit(train_images, train_labels, epochs=10, validation_split = 0.1, callbacks=[es])
        test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
        
        return test_loss, test_acc

In [26]:
# rand_run_list = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900]
rand_run_list = [0]
for count, random_state in enumerate(rand_run_list):
    cnn_bo = BayesianOptimization(bo_tune_cnn, hyperspace_cnn, random_state=0)
    cnn_bo.maximize(init_points=40, n_iter=20,acq='ucb')
    best_params = cnn_bo.max['params']

    bo_val_loss = []
    for i,res in enumerate(cnn_bo.res):
        bo_val_loss.append(-1.0 * cnn_bo.res[i]['target'])    

    loss = np.array(bo_val_loss)
    loss_minimum_accumulate = np.minimum.accumulate(loss)
    df = pd.DataFrame(loss_minimum_accumulate).transpose()
    df.to_csv("BO_tune_cnn.csv", mode = 'a', index = False, header = False)

    eval_result = run_cnn(best_params)
    df_test = pd.DataFrame(eval_result).transpose()
    df_test.to_csv("BO_test_cnn.csv", mode = 'a', index = False, header = False)

|   iter    |  target   |  beta_1   |  beta_2   | conv1_act | conv1_... | conv1_ker | conv2_act | conv2_... | conv2_ker | conv3_act | conv3_... | conv3_ker | dense1... | dense1... | dense2... | dense2... | dropou... | learni... | pool1_ker | pool2_ker | pool3_ker |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/2
1407/1407 [==============================] - 141s 100ms/step - loss: 1.5801 - accuracy: 0.4202 - val_loss: 1.3396 - val_accuracy: 0.5242
Epoch 2/2
1407/1407 [==============================] - 138s 98ms/step - loss: 1.2027 - accuracy: 0.5688 - val_loss: 1.0878 - val_accuracy: 0.6144
|  1        | -1.088    |  0.9549   |  0.9858   |  4.219    |  26.54    |  5.389    |  4.521    |  25.25    |  9.134    |  6.746    |  69.17    |  8.334    |  3.702    |  50.1

KeyboardInterrupt: ignored